In [1]:
import pyspark
from pyspark.sql import SparkSession, types

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

your 131072x1 screen size is bogus. expect trouble
24/03/04 22:28:30 WARN Utils: Your hostname, DESKTOP-IQOU5OM resolves to a loopback address: 127.0.1.1; using 172.20.246.72 instead (on interface eth0)
24/03/04 22:28:30 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/04 22:28:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark.version

'3.5.0'

In [7]:
import pandas as pd

In [8]:
df_fhv = pd.read_csv('data/raw/fhv/2019/10/fhv_tripdata_2019-10.csv.gz')

In [11]:
df_fhv_copy = df_fhv.head(10)

In [13]:
spark.createDataFrame(df_fhv_copy).schema

StructType([StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropOff_datetime', StringType(), True), StructField('PUlocationID', DoubleType(), True), StructField('DOlocationID', DoubleType(), True), StructField('SR_Flag', DoubleType(), True), StructField('Affiliated_base_number', StringType(), True)])

In [15]:
fhv_schema = types.StructType([
    types.StructField('dispatching_base_num', types.StringType(), True), 
types.StructField('pickup_datetime', types.StringType(), True), 
types.StructField('dropOff_datetime', types.StringType(), True), 
types.StructField('PUlocationID', types.IntegerType(), True), 
types.StructField('DOlocationID', types.IntegerType(), True), 
types.StructField('SR_Flag', types.DoubleType(), True), 
types.StructField('Affiliated_base_number', types.StringType(), True)
])

In [17]:
df_fhv_result = spark.read \
                .option("header","true") \
                .schema(fhv_schema) \
                .csv('./data/raw/fhv/2019/10/*')

In [18]:
df_fhv_result.show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropOff_datetime|PUlocationID|DOlocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B00009|2019-10-01 00:23:00|2019-10-01 00:35:00|         264|         264|   NULL|                B00009|
|              B00013|2019-10-01 00:11:29|2019-10-01 00:13:22|         264|         264|   NULL|                B00013|
|              B00014|2019-10-01 00:11:43|2019-10-01 00:37:20|         264|         264|   NULL|                B00014|
|              B00014|2019-10-01 00:56:29|2019-10-01 00:57:47|         264|         264|   NULL|                B00014|
|              B00014|2019-10-01 00:23:09|2019-10-01 00:28:27|         264|         264|   NULL|                B00014|
|     B00021         |2019-10-01 00:00:4

In [19]:
output_path = f'data/pq/fhv/2019/10'

In [21]:
df_fhv_result.repartition(6).write.parquet(output_path)

Record count on the 15th of October